# Connectivity Status Assessment and Action Plan

```{include} /content/ecoattributes-md.md
:start-after: "# Connectivity Status Assessment and Action Plan"
:end-before: "# Barrier Types"
```

In [1]:
import requests
import json
import pandas

def barrier_extent(barrier_type):

    request = 'https://features.hillcrestgeo.ca/bcfishpass/functions/postgisftw.wcrp_barrier_extent/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    blocked_km = result[0]['all_habitat_blocked_km']
    blocked_pct = result[0]['all_habitat_blocked_pct']

    return blocked_km, blocked_pct

def barrier_count(barrier_type):
    request = 'https://features.hillcrestgeo.ca/bcfishpass/functions/postgisftw.wcrp_barrier_count/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    n_passable = result[0]['n_passable']
    n_barrier = result[0]['n_barrier']
    n_potential = result[0]['n_potential']
    n_unknown = result[0]['n_unknown']

    sum_bar = (n_passable, n_barrier, n_potential, n_unknown)

    return n_passable, n_barrier, n_potential, n_unknown, sum(sum_bar)

def barrier_severity(barrier_type):

    request = 'https://features.hillcrestgeo.ca/bcfishpass/functions/postgisftw.wcrp_barrier_severity/items.json?watershed_group_code=HORS&barrier_type=' + barrier_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    n_assessed_barrier = result[0]['n_assessed_barrier']
    n_assess_total = result[0]['n_assess_total']
    pct_assessed_barrier = result[0]['pct_assessed_barrier']

    return n_assessed_barrier, n_assess_total, pct_assessed_barrier

def watershed_connectivity(habitat_type):

    request = 'https://features.hillcrestgeo.ca/bcfishpass/functions/postgisftw.wcrp_watershed_connectivity_status/items.json?watershed_group_code=HORS&barrier_type=' + habitat_type

    response_api = requests.get(request)
    parse = response_api.text
    result = json.loads(parse)

    connect_stat = result[0]['connectivity_status']

    return str(round(connect_stat))

In [2]:
#creating table 7
import pandas as pd
import numpy as np
import matplotlib as mpl
import warnings

warnings.filterwarnings('ignore')

df = pd.DataFrame({"Target Species":["Andromous Salmon"," "],
                   "KEA":["Available Habitat"," "],
                   "Indicator":["% of total linear habitat","Current Status:"],
                   "Poor":["<80%"," "],
                   "Fair":["  "," "],
                   "Good":["81-90%"," "],
                   "Very Good":[">90%", watershed_connectivity("ALL")]
                   })


def highlighttab7(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="<80%" : color = red
    elif val[0:].isdigit() and int(val) < 80 : color = red
    elif val=="  ": color = yellow
    elif val=="81-90%"  : color = lgreen
    elif val[0:].isdigit() and (int(val) >= 80 and int(val) < 90) : color = lgreen 
    elif val ==">90%": color = dgreen
    elif val[0:].isdigit() and int(val) >= 90 : color = dgreen 
    elif val == "Current Status:" : return "font-weight: bold"
    else: color = 'white'
    return 'background-color: %s' % color

df.style.applymap(highlighttab7).hide_index()



Target Species,KEA,Indicator,Poor,Fair,Good,Very Good
Andromous Salmon,Available Habitat,% of total linear habitat,<80%,,81-90%,>90%
,,Current Status:,,,,93


**Comments**: Indicator rating definitions are based on the consensus decisions of the planning team, including the decision not to define Fair. The current status is based on the CWF Barrier Prioritization Model output, which is current as of March 2022.

In [3]:
#creating table 7
import pandas as pd
import numpy as np
import matplotlib as mpl
import warnings
from myst_nb import glue

warnings.filterwarnings('ignore')

df = pd.DataFrame({"Target Species":["Andromous Salmon",""],
                   "KEA":["Available Overwintering Habitat",""],
                   "Indicator":["Total Area (m2) of overwintering habitat accessible","Current Status:"],
                   "Poor":["?",""],
                   "Fair":[" ?",""],
                   "Good":["? ",""],
                   "Very Good":[" ? ",""]
                   })

def highlighttab7b(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="?" : color = red
    elif val[0:].isdigit() and int(val) < 80 : color = red
    elif val==" ?": color = yellow
    elif val=="? "  : color = lgreen
    elif val[0:].isdigit() and (int(val) >= 80 and int(val) < 90) : color = lgreen 
    elif val ==" ? ": color = dgreen
    elif val[0:].isdigit() and int(val) >= 90 : color = dgreen 
    elif val == "Current Status:" : return "font-weight: bold"
    else: color = 'white'
    return 'background-color: %s' % color

data = df.style.applymap(highlighttab7b).hide_index()

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

glue("Table7", data)


Target Species,KEA,Indicator,Poor,Fair,Good,Very Good
Andromous Salmon,Available Overwintering Habitat,Total Area (m2) of overwintering habitat accessible,?,?,?,?
,,Current Status:,,,,


**Comments:** No baseline data exists on the extent of overwintering habitat in the watershed. A priority action is included in the Operational Plan (strategy 2.3) to develop a habitat layer, and this will be used to inform this connectivity status assessment in the future.

```{glue:figure} Table7
:name: "table7"

*Connectivity status assessment for (a) linear habitat (spawning and rearing) and (b) overwintering habitat in the Horsefly River watershed. The Available Habitat KEA is evaluated by dividing the length of linear habitat that is currently accessible to target species by the total length of all linear habitat in the watershed. The Available Overwintering Habitat KEA is evaluated as the sum of all areal overwintering habitat that is accessible to target species.*
```

# Barrier Types

```{include} /content/ecoattributes-md.md
:start-after: "# Barrier Types"
:end-before: "### Small Dams (<3 m height)"
```

In [4]:
from ipywidgets import *
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

#condition
def condition(pct):
    rating = ""
    if pct < 30 : rating = "Low"
    elif (pct >= 30) and (pct < 71) : rating = "Medium"
    elif (pct >= 71) and (pct < 90) : rating = "High"
    else : rating = "Very High"
    return rating

#rating classifier
def rating(threat, barrier):
    if threat == "extent":
        if barrier == "DAM":
            pct = barrier_extent(barrier)[1]
            rating = condition(pct)
        elif barrier == "ROAD":
            pct = barrier_extent('ROAD, RESOURCE/OTHER')[1] + barrier_extent('ROAD, DEMOGRAPHIC')[1]
            rating = condition(pct)
    elif threat == "severity":
        if barrier == "DAM":
            pct = barrier_severity(barrier)[2]
            rating = condition(pct)
        elif barrier == "ROAD":
            pct = barrier_severity('ROAD, RESOURCE/OTHER')[2] + barrier_severity('ROAD, DEMOGRAPHIC')[2]
            rating = condition(pct)
            
    return rating
            

        




df = pd.DataFrame({"Barrier Types":["Road-Stream Crossings","Lateral Barriers","Small Dams(<3m height)","Trail-stream Crossings", "Natural Barriers"],
                   "Extent":[rating("extent", "ROAD"),"High",rating("extent", "DAM"), "Low", "Medium"],
                   "Severity":[rating("severity", "ROAD"),"Very High",rating("severity", "DAM"), "Low", "High"],
                   "Irreversibility":["Medium","High","High", "Medium", "Low"],
                   "Overall Threat Rating:":["Very High","High","Medium", "Low", "Low"]
                   }).style.set_properties(subset=["Overall Threat Rating:"], **{'font-weight': 'bold'})

def highlight(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="Very High": color = red
    elif val=="High": color = yellow
    elif val=="Medium": color = lgreen
    elif val =="Low": color = dgreen
    else: color = 'white'
    return 'background-color: %s' % color

#df = df.style.set_properties(subset=["Overall Threat Rating"], **{'font-weight': 'bold'})

data = df.applymap(highlight).hide_index()

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

glue("Table8", data)




Barrier Types,Extent,Severity,Irreversibility,Overall Threat Rating:
Road-Stream Crossings,High,Very High,Medium,Very High
Lateral Barriers,High,Very High,High,High
Small Dams(<3m height),Low,Medium,High,Medium
Trail-stream Crossings,Low,Low,Medium,Low
Natural Barriers,Medium,High,Low,Low


```{glue:figure} Table8
:name: "table8"

*Barrier Types in the Horsefly River watershed and barrier rating assessment results. For each barrier type listed, "Extent refers to the proportion of anadromous salmon habitat that is being blocked by that barrier type, "Severity" is the proportion of structures for each barrier type that are known to block passage for target species based on field assessments, and "Irreversibility" is the degree to which the effects of a barrier type can be reversed and connectivity restored. The amount of habitat blocked used in this exercise is a representation of total amount of combined spawning and rearing habitat. All ratings in this table have been updated from version 1.0 to version 2.0 of the Horsefly River Watershed Connectivity Remediation Plan based on the most recent field assessments.*
```

In [5]:
import requests
import json
import pandas
from myst_nb import glue



#glue class for variables to allow embedding in markdown
glue("dam_km", barrier_extent('DAM')[0])
glue("dam_pct", round(barrier_extent('DAM')[1]))
glue("total_barrier", barrier_severity('DAM')[1])



8.09

23

9

### Small Dams (<3 m height)

In [6]:
#glue class for variables to allow embedding in markdown
glue("resource_km", barrier_extent('ROAD, RESOURCE/OTHER')[0])
glue("resource_pct", round(barrier_extent('ROAD, RESOURCE/OTHER')[1]))
glue("demo_km", barrier_extent('ROAD, DEMOGRAPHIC')[0])
glue("demo_pct", round(barrier_extent('ROAD, DEMOGRAPHIC')[1]))
glue("resource_sev", round(barrier_severity('ROAD, RESOURCE/OTHER')[2]))
glue("demo_sev", round(barrier_severity('ROAD, DEMOGRAPHIC')[2]))

sum_road = (barrier_severity('ROAD, RESOURCE/OTHER')[1], barrier_severity('ROAD, DEMOGRAPHIC')[1])


glue("sum", sum(sum_road))

19.57

56

7.31

21

60

73

244

```{include} /content/ecoattributes-md.md
:start-after: "### Small Dams (<3 m height)"
:end-before: "# Goals"
```


```{figure} figures/figure3.png
---
height: 400px
width: 1000px
name: directive-fig
---
*Situation analysis developed by the planning team to identify factors that contribute to fragmentation (orange boxes), biophysical results (brown boxes), and potential strategies/actions to improve connectivity (yellow hexagons) for target species in the Horsefly River watershed.*
```

# Goals


In [7]:
#creating table 7
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

df = pd.DataFrame({"Goal #": [1,2],
                   "Goal": ["By 2040, the percent (%) of total linear habitat accessible to anadromous salmon will increase from 94% to 96% within the Horsefly River watershed (i.e., reconnect at least 11.7 km of habitat).",
                            "By 2024, the total area of overwintering habitat accessible to Anadromous Salmon will increase by 1,500 m2 within the Horsefly River watershed. "]
                    })

data = df.style.hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

glue("Table9", data)


Goal #,Goal
1,"By 2040, the percent (%) of total linear habitat accessible to anadromous salmon will increase from 94% to 96% within the Horsefly River watershed (i.e., reconnect at least 11.7 km of habitat)."
2,"By 2024, the total area of overwintering habitat accessible to Anadromous Salmon will increase by 1,500 m2 within the Horsefly River watershed."


```{glue:figure} Table9
:name: "table9"

*Goals to improve (1) spawning and rearing and (2) overwintering habitat connectivity for target species in the Horsefly River watershed over the lifespan of the WCRP (2021-2040). The goals were established through discussions with the planning team and represent the resulting desired state of connectivity in the watershed. The goals are subject to change as more information and data are collected over the course of the plan timeline (e.g., the current connectivity status is updated based on barrier field assessments).*
```

```{include} /content/ecoattributes-md.md
:start-after: "# Goals"
:end-before: "## Strategy 1: Crossing Remediation"
```



## Strategy 1: Crossing Remediation


In [8]:
import numpy as np
from IPython.display import display
import pandas as pd

data = pd.read_csv('tables/Strategy1.csv', index_col=False, skip_blank_lines=False )
 
def fix_table(val):
    return str(val)

def highlighttab7(val):
    red = '#ff0000;'
    yellow = '#ffff00;'
    lgreen = '#92d050;'
    dgreen = '#03853e;'

    if val=="Medium" or val=="Need more information": color = yellow
    elif val=="Very high" or val=="Very effective" : color = lgreen
    elif val =="High" or val=="Effective": color = dgreen
    else: color = 'white'
    return 'background-color: %s' % color

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})
data = data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

display(data)


ID,Actions,Details,Feasibility,Impact,Effectiveness
1.1,Remediate crossings that are acting as barriers,"This action represents some projects that would be led by the planning team with conservation funds (e.g., orphaned barriers or those owned by individuals), while other remediation projects would be the responsibility of the barrier owner. Industry will have to be engaged to successfully implement this intervention. PSC Southern Boundary Restoration and Enhancement Fund proposal: - Complete remediation of one priority barrier, including engineering designs HCTF proposal: - Complete remediation of one priority barrier CNFASAR proposal (2022-26): - Complete remediation of one priority barrier per year for four years HRR Can help with finding local people to implement remediation projects.",High,Very high,Effective
1.2,Lobby that the government enforce their regulations,"This can apply to both provincial and federal governments. For example, advocating for increased discretionary decisions to remove barriers to fish. One action could be to submit barrier assessment data to show proof that regulations are not being followed.",Very high,High,Effective
1.3,Initiate a barrier owner outreach program for locations on the barrier remediation shortlist,"Work with landowners / users (e.g., ATV groups) to identify and remediate their aquatic barriers. Education component can help prevent barriers in the first place. HRR to reach out to owners of confirmed barriers to discuss remediation options; CWF to reach out to provincial representatives.",Very high,Very high,Very effective
1.4,Knowledge Gap: Continue updating the barrier prioritization model,The model has been updated to reflect 2021 field assessments and intermediate barrier review.,Very high,High,Effective
1.5,Knowledge Gap: conduct field assessments on updated preliminary barrier list using the provincial fish passage framework and update connectivity goal if additional barriers are added to the barrier remediation shortlist,Twenty-six field assessments performed in 2021.,Very high,Very high,Very effective
1.6,Update longitudinal connectivity goal if additional barriers are added to the barrier remediation shortlist,,,,
1.7,Knowledge Gap: Identify and map crossing ownership,For barriers on the barrier remediation shortlist.,Very high,Very high,Very effective
1.8,Knowledge Gap: Compile road maintenance schedules,"Ground-truthing is important, as the schedules do not always reflect what happens in the field.",High,High,Effective
1.9,Knowledge Gap: Survey trail-stream crossings to confirm low pressure rating values,"Need to access detailed trail maps in the watershed to prioritize our time and resources. This should be accomplished as people are out surveying for other reasons rather than spending time and resources specifically to fill this knowledge gap. CNFASAR proposal: Collaborate with WLFN to: - Develop field assessment protocols for whether ATV trail stream crossings pass fish, and for assessing other effects on fish habitat - Map potential trail-stream crossings on salmon habitat that could be assessed - Assess 30-50 trail stream crossings, record measurements, and take pictures",Very high,Medium,Need more information
,,,,,



## Strategy 2: Lateral Barrier Remediation	


In [9]:
data = pd.read_csv('tables/Strategy2.csv', escapechar='\n', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])


ID,Actions,Details,Feasibility,Impact,Effectiveness
2.1,Remediate dikes / berms / other lateral barriers,,High,Very high,Effective
2.2,Initiate a barrier owner outreach program,,Very high,Very high,Very effective
2.3,"Knowledge Gap: Identify and map year-round lateral habitat, as well as overwintering habitat","Explore the use of a drone to identify lateral habitat. - Volunteers from the HRR will conduct field habitat assessments following modules in the Pacific Streamkeepers Handbook to assess disconnected lateral and overwintering salmon habitats in the Horsefly watershed CNFASAR proposal: -Funding for equipment in 2022-2023, and for field transportation in 2022-2023, 2023-2024",Very high,Very high,Very effective
2.4,Knowledge Gap: Map lateral barriers and barrier ownership,Focus on identifying ownership of priority lateral barriers that we want to remediate in the short-term.,Very high,Very high,Very effective
2.5,Knowledge Gap: Develop a framework to assess and prioritize between different lateral barrier remediation projects,"CWF is leading a provincial-scale analysis of the effect of rail lines on connectivity for Anadromous Salmonids, as part of this project lateral habitat and barrier assessments and prioritization methods will be developed.",Very high,Very high,Very effective


## Strategy 3: Dam Remediation


In [10]:
data = pd.read_csv('tables/Strategy3.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])


ID,Actions,Details,Feasibility,Impact,Effectiveness
3.1,Remediate Dams,,Medium,Very high,Need more information
3.2,Install Fish Passage,,Medium,High,Need more information
3.3,Connect with Cattleman's Association to explore a partnership to remediate dams,This may involve exploring alternative water management actions that would allow for the remediation of irrigation dams.,High,Medium,Need more information
3.4,Knowledge Gap: Continue updating the barrier prioritization model,The model has been updated to reflect 2021 field assessments and intermediate barrier review.,Very high,High,Effective
3.5,Knowledge Gap: Assess dams to determine whether they exist and are truly blocking fish habitat,Four dams were assessed during 2021 field season; additional field assessment needed.,Very high,High,Effective
3.6,Knowledge Gap: Identify and map dam ownership,,Very high,Very high,Very effective


## Strategy 4: Barrier Prevention


In [11]:
import pandas as pd

data = pd.read_csv('tables/Strategy4.csv', index_col=False)




data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])


ID,Actions,Details,Feasibility,Impact,Effectiveness
4.1,Explore potential partnerships with industrial companies,Invite industrial players to a workshop on how to apply crossing / lateral barrier BMPs. BMPs could include those that minimize the need for road-stream crossings.,Very high,High,Effective
4.2,Stabilize sediment sources that are explicitly linked to sediment wedges or erosion that are acting as barriers,"This could include numerous bank stabilization techniques, including restoring riparian vegetation. This applies to some tributaries that have altered confluence areas - the link needs to be made between confluence alterations and timing of movement for juvenile fish. Local ranchers and Cattleman's association could be engaged, as well as forestry licensees.",Very high,Medium,Need more information


## Strategy 5: Communication and Education


In [12]:
data = pd.read_csv('tables/Strategy5.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

data = data.applymap(fix_table)

data = data.style.applymap(highlighttab7).hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

ID,Actions,Details
5.1,Implement the WCRP Progress Tracking Plan,The WCRP Progress Tracking Plan will help the team determine if we are achieving our goals and objectives.
5.2,Develop a communication strategy to raise awareness and support for this WCRP,"This intervention includes communicating both the WCRP and the collaborative process in developing it, as well as communicating outcomes (e.g., barrier remediations). CNFASAR proposal: - HRR will work with CWF to develop outreach and communications materials, including press releases, social media content, a video, and content for their website - With HRR, CWF will present on fish passage issues and solutions at the annual Horsefly River Salmon Festival"


```{include} /content/ecoattributes-md.md
:start-after: "## Strategy 5: Communication and Education"
:end-before: "# Operational Plan"
```


```{figure} figures/figure4.png
---
height: 400px
width: 1000px
name: fig4
---
*Theory of change developed by the planning team for the actions identified under Strategy 1: Crossing Remediation in the Horsefly River watershed.*
```
```{figure} figures/figure5.png
---
height: 400px
width: 1000px
name: fig5
---
*Theory of change developed by the planning team for the actions identified under Strategy 2: Lateral Barrier Remediation in the Horsefly River watershed.*
```
```{figure} figures/figure6.png
---
height: 400px
width: 1000px
name: fig6
---
*Theory of change developed by the planning team for the actions identified under Strategy 3: Dam Remediation in the Horsefly River watershed.*
```
```{figure} figures/figure7.png
---
height: 400px
width: 1000px
name: fig7
---
*Theory of change developed by the planning team for the actions identified under Strategy 4: Barrier Prevention in the Horsefly River watershed.*
```


# Operational Plan

```{include} /content/ecoattributes-md.md
:start-after: "# Operational Plan"
:end-before: "# Funding Sources"
```

In [13]:
from IPython.display import display
import pandas as pd
import numpy as np

def df_operation(val):
    return "background-color: black; color: white"


data = pd.read_csv('tables/Table13.csv', index_col=False)

data = data.replace(np.nan, '', regex=True)

rows = pd.IndexSlice[[0,10,16,23,26,29,30,31], :]

data = data.style.applymap(df_operation, subset=rows).hide_index().set_properties(**{'text-align': 'left'})
data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

glue("Table13", data)

Strategy / Actions,Lead(s) [1],Participants3,Total Budget
Strategy 1: Crossing Remediation,,,"$3,666,300.00"
1.1 – Remediate crossings that are acting as barriers,CWF,"Horsefly River Roundtable, Fisheries and Oceans Canada (DFO)","$3,500,000.00"
1.2 – Lobby that the government enforce their regulations,TBD,"CWF, Horsefly River Roundtable, Williams Lake First Nation (WLFN)","$10,000.00"
1.3 – Initiate a barrier owner outreach program for locations on the barrier remediation shortlist,"HRR, CWF, DFO",,TBD
1.4 – Knowledge Gap: Continue updating the barrier prioritization model,CWF,TBD,"$100,000.00"
1.5 – Knowledge Gap: conduct field assessments on updated preliminary barrier list using the provincial fish passage framework and update connectivity goal if additional barriers are added to the barrier remediation shortlist,CWF,"Horsefly River Roundtable, DFO","$50,300.00"
1.6 - Update longitudinal connectivity goal if additional barriers are added to the barrier remediation shortlist,,,
1.7 – Knowledge Gap: Identify and map crossing ownership for barriers on the barrier remediation shortlist,TBD,"CWF, DFO (Anthonie)","$1,500.00"
1.8 – Knowledge Gap: Compile road maintenance schedules,DFO,"CWF, WLFN, DFO, FLNRORD","$2,000.00"
1.9 – Knowledge Gap: Survey trail-stream crossings to confirm low pressure rating values,WLFN,"CWF, DFO","$2,500.00"


```{glue:figure} Table13
:name: "table13"

*Operational plan to support the implementation of strategies and actions to improve connectivity for target species in the Horsefly River watershed.*
```

# Funding Sources

In [14]:
from IPython.display import display
import pandas as pd

data = pd.read_csv('tables/Table14.csv', index_col=False)

data = data.style.hide_index().set_properties(**{'text-align': 'left'})

data.set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

glue("Table14", data)

Funding Source,Spending Restrictions and Other Consideration
Land Based Investment Strategy,Assessment and remediation of fish passage using provincial strategic approach. Primarily for remediation of Ministry-owned/orphaned barriers on forest service roads.
Environmental Enhancement Fund,"Fish and wildlife passage improvements and restoration at stream and animal crossings at MOTI roads including culvert retrofits and replacement to restore Pacific salmon and trout access, and wildlife tunnels. Primarily for crossings linked to highway infrastructure."
Community Salmon Program,"For projects supporting the protection, conservation and enhancement or rehabilitation of Pacific salmonids and their habitat. Funding for volunteer and not-for-profit community-based groups. Applicant must have a significant volunteer component to their group and to the project. Requires 50% match for funding (volunteer, in-kind, donation or other grants)."
Southern Boundary Restoration and Enhancement Fund,"Supports 3 activities: (1) develop improved information for resource management; (2) Rehabilitate and restore marine and freshwater habitat; and (3) enhance wild stock production through low technology techniques. Emphasis for funding is on stocks of conservation concern, particularly those contributing to a fishery and stocks of bilateral fishery relevance."
Habitat Conservation Trust Foundation Enhancement and Restoration Grants,"Projects that focus on freshwater wild fish, native wildlife species and their habitats, have the potential to achieve a significant conservation outcome, while maintaining or enhancing opportunities for fishing, hunting, trapping, wildlife viewing and associated outdoor recreational activities. Primary focus is on provincially managed fisheries such as Steelhead and Westslope Cutthroat Trout. Requires 50% funding match."
Environmental Damages Fund,"Direct funds received from fines, court orders and voluntary payments to priority projects that will benefit Canada’s natural environment, under 4 categories of improvement (in order of preference): (1) restoration, (2) environmental quality improvement, (3) research and development, and (4) education and awareness."
Habitat Stewardship Program for Aquatic Species at Risk,"Program for non-profits, Indigenous governments, academic institutions for activities that align with recovery actions identified in SARA recovery documents and/or COSEWIC assessment documents. Project must address one or more of 3 broad categories: (1) Important habitat for aquatic species at risk is improved and/or managed to meet their recovery needs; (2) Threats to aquatic species at risk and/or their habitat are stopped, removed, and/or mitigated; (3) Collaboration and partnerships support the conservation and recovery of aquatic species at risk. Limited to at-risk species listed under COSEWIC and/or SARA as threatened, endangered or special concern."
Canada Nature Fund for Aquatic Species at Risk,"Funding program aimed at addressing priority threats for aquatic species at risk listed as endangered, threatened or Special Concern by COSEWIC, as they align with existing federal, provincial or other local recovery plans. Limited to species in the Columbia and Fraser basins in BC, among other priority areas across Canada. Focus on multi-year, multi-partner initiatives that apply an ecosystem or multi-species approach and create a legacy by enabling recovery actions that carry beyond the life of the funding program. Amounts from $100K-$1M available per year."
BC Salmon Restoration and Innovation Fund,"Funding for Indigenous enterprises, academia, industry associations, stewardship groups and commercial groups to support initiatives that support the protection and restoration of wild Pacific salmon and other BC fish stocks or ensure fish and seafood sector in BC is environmentally and economically sustainable. Five main priorities including species of concern rebuilding through habitat restoration with prio

```{glue:figure} Table14
:name: "table14"

*Potential funding sources for plan implementation in the Horsefly River watershed. The Canadian Wildlife Federation and the planning team can coordinate proposal submission through these sources.*
```